In [1]:
%cd ..

/Users/treycole/Codes/WanPy


In [2]:
from WanPy.WanPy import *
import WanPy.models as models
import WanPy.plotting as plot

from pythtb import *
import numba
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sympy as sp 
import scipy

In [3]:
delta = 1
t0 = 0.27
tprime = 0.5

model = models.chessboard(t0, tprime, delta).make_supercell([[2,0], [0,2]])

orbs = model.get_orb()
n_orb = model.get_num_orbitals()
n_occ = int(n_orb/2)
lat_vecs = model.get_lat() # lattice vectors

low_E_sites = np.arange(0, model.get_num_orbitals(), 2)
high_E_sites = np.arange(1, model.get_num_orbitals(), 2)

In [4]:
nkx = 10
nky = 10
Nk = nkx*nky

k_mesh = gen_k_mesh(nkx, nky, flat=False, endpoint=False)
k_mesh_flat = gen_k_mesh(nkx, nky, flat=True, endpoint=False)

u_wfs_Wan = wf_array(model, [nkx, nky])
for i in range(k_mesh.shape[0]):
    for j in range(k_mesh.shape[1]):
        u_wfs_Wan.solve_on_one_point(k_mesh[i,j], [i,j])
psi_wfs_Wan = get_bloch_wfs(orbs, u_wfs_Wan, k_mesh)

In [8]:
omit_sites = 2
tf_list = list(np.setdiff1d(low_E_sites, [omit_sites])) # delta on lower energy sites omitting the last site

W0, psi_til_wan = Wannierize(orbs, u_wfs_Wan, tf_list, ret_psi_til=True)
u_tilde_wan = get_bloch_wfs(orbs, psi_til_wan, k_mesh, inverse=True)

In [9]:
# outer window of entangled bands is full occupied manifold
outer_states = u_wfs_Wan._wfs[..., :n_occ, :]

In [10]:
W0_max_loc, Wf_cntrs = max_loc_Wan(lat_vecs, orbs, u_wfs_Wan, tf_list, outer_states, 
        iter_num_omega_i=100, iter_num_omega_til=100,
        state_idx=None, verbose=True, return_uwfs=False, return_wf_centers=True, eps=2e-3
        )

0 Omega_I: 30.715446448258078
1 Omega_I: 1.2438251657362418
2 Omega_I: 0.7349167867984816
3 Omega_I: 0.6986940861114543
4 Omega_I: 0.686796364769752
5 Omega_I: 0.632961641263547
6 Omega_I: 0.626636311272309
7 Omega_I: 0.631710768512534
8 Omega_I: 0.6003457814701855
9 Omega_I: 0.5977244363935127
10 Omega_I: 0.6028464247360443
11 Omega_I: 0.5812924019703946
12 Omega_I: 0.5794159201819872
13 Omega_I: 0.5839553942671607
14 Omega_I: 0.5680602566650421
15 Omega_I: 0.5664664320730123
16 Omega_I: 0.5703606438141883
17 Omega_I: 0.5581658152911017
18 Omega_I: 0.5567350660992091
19 Omega_I: 0.560083700684101
20 Omega_I: 0.5504405755005912
21 Omega_I: 0.549148150107785
22 Omega_I: 0.5520548171417803
23 Omega_I: 0.5442439940026563
24 Omega_I: 0.5430795580937798
25 Omega_I: 0.5456331108737755
26 Omega_I: 0.5391793578700099
27 Omega_I: 0.5381340420604892
28 Omega_I: 0.5404049686349754
29 Omega_I: 0.5349822540727419
30 Omega_I: 0.5340463502618605
31 Omega_I: 0.5360892306247429
32 Omega_I: 0.5314667549

In [8]:
# Wan_idx = 0
# plot.plot_Wan(W0_max_loc, Wan_idx, orbs, lat_vecs, plot_decay=True, show=True)

In [13]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [9]:
%lprun -f max_loc_Wan max_loc_Wan(lat_vecs, orbs, u_wfs_Wan, tf_list, outer_states, iter_num_omega_i=100, iter_num_omega_til=100,state_idx=None, verbose=True, return_uwfs=False, return_wf_centers=True, eps=2e-3)

UsageError: Line magic function `%lprun` not found.


In [16]:
%lprun -f get_max_loc_uwfs get_max_loc_uwfs(lat_vecs, orbs, u_tilde_wan, eps=1 / 160, iter_num=100, verbose=True, tol=1e-17)

0 Omega_til = 0.8151596478830426, Grad mag: 4.764035122086684
1 Omega_til = 0.8049825154708674, Grad mag: 5.59324099992118
2 Omega_til = 0.798702949222287, Grad mag: 6.117004456573842
3 Omega_til = 0.7944306450457037, Grad mag: 6.466469022138358
4 Omega_til = 0.7913432681306556, Grad mag: 6.703755934620765
5 Omega_til = 0.7890158753500601, Grad mag: 6.8639572205637025
6 Omega_til = 0.7872048158525299, Grad mag: 6.968844653353843
7 Omega_til = 0.7857599707917087, Grad mag: 7.032800898957758
8 Omega_til = 0.7845837665618963, Grad mag: 7.065763493317049
9 Omega_til = 0.7836100809182652, Grad mag: 7.074839313467839
10 Omega_til = 0.7827925689407351, Grad mag: 7.065252459882976
11 Omega_til = 0.7820978252165153, Grad mag: 7.040930547610783
12 Omega_til = 0.7815011921009792, Grad mag: 7.004882581256569
13 Omega_til = 0.7809840920947234, Grad mag: 6.959450872066421
14 Omega_til = 0.7805322759849971, Grad mag: 6.906483824460949
15 Omega_til = 0.78013464098886, Grad mag: 6.8474573691605745
16 O

Timer unit: 1e-09 s

Total time: 3.62316 s
File: /Users/treycole/Codes/WanPy/WanPy/WanPy.py
Function: get_max_loc_uwfs at line 884

Line #      Hits         Time  Per Hit   % Time  Line Contents
   884                                           @profile
   885                                           def get_max_loc_uwfs(
   886                                               lat_vecs, orbs, u_wfs, eps=1 / 160, iter_num=10, verbose=False, tol=1e-17
   887                                           ):
   888                                           
   889         1       4000.0   4000.0      0.0      if isinstance(u_wfs, wf_array):
   890                                                   shape = u_wfs._wfs.shape  # [*nks, idx, orb]
   891                                               else:
   892         1       3000.0   3000.0      0.0          shape = u_wfs.shape  # [*nks, idx, orb]
   893         1       1000.0   1000.0      0.0      nks = shape[:-2]  # tuple defining number of k poin

In [18]:
M = k_overlap_mat(lat_vecs, orbs, u_tilde_wan)  # [kx, ky, b, m, n]
%lprun -f find_min_unitary find_min_unitary(lat_vecs, M, iter_num=100, eps=1e-3, verbose=True, tol=1e-17)

0 Omega_til = 0.8317740800361155, Grad mag: 4.764035122086684
1 Omega_til = 0.8283172871060959, Grad mag: 4.9009743312798575
2 Omega_til = 0.8252158821440498, Grad mag: 5.029426612699307
3 Omega_til = 0.8224200612882975, Grad mag: 5.150013940852343
4 Omega_til = 0.8198884918135826, Grad mag: 5.263303937376195
5 Omega_til = 0.8175866795770549, Grad mag: 5.36981479114968
6 Omega_til = 0.8154856785372483, Grad mag: 5.47001985102588
7 Omega_til = 0.8135610663442409, Grad mag: 5.56435185942985
8 Omega_til = 0.8117921276288159, Grad mag: 5.653206821324989
9 Omega_til = 0.8101612000213269, Grad mag: 5.736947518626945
10 Omega_til = 0.8086531481462453, Grad mag: 5.8159066889855024
11 Omega_til = 0.8072549386461924, Grad mag: 5.890389892434203
12 Omega_til = 0.8059552952749766, Grad mag: 5.960678091298843
13 Omega_til = 0.804744417699389, Grad mag: 6.0270299689734514
14 Omega_til = 0.8036137511969582, Grad mag: 6.089684012376241
15 Omega_til = 0.8025557971792618, Grad mag: 6.148860381516608
16 

Timer unit: 1e-09 s

Total time: 3.76989 s
File: /Users/treycole/Codes/WanPy/WanPy/WanPy.py
Function: find_min_unitary at line 785

Line #      Hits         Time  Per Hit   % Time  Line Contents
   785                                           @profile
   786                                           def find_min_unitary(lat_vecs, M, eps=1 / 160, iter_num=10, verbose=False, tol=1e-17):
   787                                           
   788         1       2000.0   2000.0      0.0      shape = M.shape
   789         1       1000.0   1000.0      0.0      nks = shape[:-3]
   790         1       1000.0   1000.0      0.0      dim_k = len(nks)
   791         1      40000.0  40000.0      0.0      Nk = np.prod(nks)
   792         1          0.0      0.0      0.0      numNN = shape[-3]
   793         1          0.0      0.0      0.0      num_state = shape[-1]
   794         2      12000.0   6000.0      0.0      k_idx_arr = list(
   795         1       6000.0   6000.0      0.0          product

In [19]:
%lprun -f spread_recip spread_recip(lat_vecs, M, decomp=True)

Timer unit: 1e-09 s

Total time: 0.035643 s
File: /Users/treycole/Codes/WanPy/WanPy/WanPy.py
Function: spread_recip at line 453

Line #      Hits         Time  Per Hit   % Time  Line Contents
   453                                           @profile
   454                                           def spread_recip(lat_vecs, M, decomp=False):
   455                                               """
   456                                               Args:
   457                                                   M (np.array):
   458                                                       overlap matrix
   459                                                   decomp (bool, optional):
   460                                                       Whether to compute and return decomposed spread. Defaults to False.
   461                                           
   462                                               Returns:
   463                                                   spread | [spr

In [92]:
def spread_recip(lat_vecs, M, decomp=False):
    """
    Args:
        M (np.array):
            overlap matrix
        decomp (bool, optional):
            Whether to compute and return decomposed spread. Defaults to False.

    Returns:
        spread | [spread, Omega_i, Omega_tilde], expc_rsq, expc_r_sq :
            quadratic spread, the expectation of the position squared,
            and the expectation of the position vector squared
    """
    shape = M.shape
    n_states = shape[3]
    nks = M.shape[:-3]
    k_axes = tuple([i for i in range(len(nks))])
    Nk = np.prod(nks)

    # Assumes only one shell for now
    k_shell, _ = get_k_shell(*nks, lat_vecs=lat_vecs, N_sh=1, tol_dp=8, report=False)
    w_b = get_weights(*nks, lat_vecs=lat_vecs, N_sh=1)[0]
    
    r_n = np.zeros((n_states, 2), dtype=complex)  # <\vec{r}>_n
    rsq_n = np.zeros(n_states, dtype=complex)  # <r^2>_n

    for n in range(n_states):
        for idx, b in enumerate(k_shell[0]):
            r_n[n, :] += -(1 / Nk) * w_b * b * np.sum(np.log(M[..., idx, n, n]).imag, axis=k_axes)
            rsq_n[n] += np.sum(
                    (1 / Nk) * w_b
                    * (1 - abs(M[..., idx, n, n]) ** 2
                       + np.log(M[..., idx, n, n]).imag ** 2
                       ), axis=k_axes
                )

    expc_rsq = np.sum(rsq_n)  # <r^2>
    expc_r_sq = np.sum([np.vdot(r_n[n, :], r_n[n, :]) for n in range(r_n.shape[0])])  # <\vec{r}>^2
    spread = expc_rsq - expc_r_sq
    if decomp:
        Omega_i = 0
        Omega_tilde = 0
        for idx, b in enumerate(k_shell[0]):
            Omega_i += w_b * n_states
            for n in range(n_states):
                Omega_tilde += np.sum(
                    (1 / Nk) * w_b
                    * (-np.log(M[..., idx, n, n]).imag - np.vdot(b, r_n[n])) ** 2, 
                    axis=k_axes
                )
                for m in range(n_states):
                    Omega_i -= np.sum((1 / Nk) * w_b * abs(M[..., idx, m, n]) ** 2, axis=k_axes)
                    if m != n:
                        Omega_tilde += np.sum((1 / Nk) * w_b * abs(M[..., idx, m, n]) ** 2, axis=k_axes)

        return [spread, Omega_i, Omega_tilde], r_n, rsq_n

    else:
        return spread, r_n, rsq_n

In [93]:
spread_recip(lat_vecs, M, decomp=True)

([(1.5077789669733317+0j), 0.6721323571565483, (0.8356466098167853+0j)],
 array([[9.82134187e-04+0.j, 1.62984779e-02+0.j],
        [9.82134187e-04+0.j, 1.01629848e+00+0.j],
        [1.00098213e+00+0.j, 1.62984779e-02+0.j],
        [1.00098213e+00+0.j, 1.01629848e+00+0.j]]),
 array([0.37721135+0.j, 1.4098083 +0.j, 1.37917562+0.j, 2.41177257+0.j]))

In [80]:
spread_recip(lat_vecs, M, decomp=True)

([(1.5077789669733281+0j), 0.6721323571565565, (0.8356466098167673+0j)],
 array([[9.82134187e-04+0.j, 1.62984779e-02+0.j],
        [9.82134187e-04+0.j, 1.01629848e+00+0.j],
        [1.00098213e+00+0.j, 1.62984779e-02+0.j],
        [1.00098213e+00+0.j, 1.01629848e+00+0.j]]),
 array([0.37721135+0.j, 1.4098083 +0.j, 1.37917562+0.j, 2.41177257+0.j]))

In [94]:
%lprun -f spread_recip spread_recip(lat_vecs, M, decomp=True)

Timer unit: 1e-09 s

Total time: 0.002922 s
File: /var/folders/nn/m4t491h92ss8vwl56z761h6c0000gn/T/ipykernel_27117/2773637555.py
Function: spread_recip at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def spread_recip(lat_vecs, M, decomp=False):
     2                                               """
     3                                               Args:
     4                                                   M (np.array):
     5                                                       overlap matrix
     6                                                   decomp (bool, optional):
     7                                                       Whether to compute and return decomposed spread. Defaults to False.
     8                                           
     9                                               Returns:
    10                                                   spread | [spread, Omega_i, Omega_tild

In [79]:
%lprun -f spread_recip spread_recip(lat_vecs, M, decomp=True)

Timer unit: 1e-09 s

Total time: 0.033473 s
File: /var/folders/nn/m4t491h92ss8vwl56z761h6c0000gn/T/ipykernel_27117/2516527829.py
Function: spread_recip at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def spread_recip(lat_vecs, M, decomp=False):
     2                                               """
     3                                               Args:
     4                                                   M (np.array):
     5                                                       overlap matrix
     6                                                   decomp (bool, optional):
     7                                                       Whether to compute and return decomposed spread. Defaults to False.
     8                                           
     9                                               Returns:
    10                                                   spread | [spread, Omega_i, Omega_tild

In [71]:

def spread_recip(lat_vecs, M, decomp=False):
    """
    Args:
        M (np.array):
            overlap matrix
        decomp (bool, optional):
            Whether to compute and return decomposed spread. Defaults to False.

    Returns:
        spread | [spread, Omega_i, Omega_tilde], expc_rsq, expc_r_sq :
            quadratic spread, the expectation of the position squared,
            and the expectation of the position vector squared
    """
    shape = M.shape
    n_states = shape[3]
    nks = M.shape[:-3]
    Nk = np.prod(np.array(nks))
    k_idx_arr = list(product(*[range(nk) for nk in nks]))  # list of all k_indices

    # Assumes only one shell for now
    k_shell, _ = get_k_shell(*nks, lat_vecs=lat_vecs, N_sh=1, tol_dp=8, report=False)
    w_b = get_weights(*nks, lat_vecs=lat_vecs, N_sh=1)[0]
    r_n = np.zeros((n_states, 2), dtype=float)  # <\vec{r}>_n
    rsq_n = np.zeros(n_states, dtype=float)  # <r^2>_n

    @njit
    def run():
        expc_rsq = 0  # <r^2>
        expc_r_sq = 0  # <\vec{r}>^2
        for n in range(n_states):
            for k in k_idx_arr:
                for idx, b in enumerate(k_shell[0]):
                    r_n[n, :] += -(1 / Nk) * w_b * b * np.log(M[k][idx, n, n]).imag
                    rsq_n[n] += (
                        (1 / Nk) * w_b
                        * (1 - abs(M[k][idx, n, n]) ** 2
                        + np.log(M[k][idx, n, n]).imag ** 2
                        )
                    )

            expc_rsq += rsq_n[n]  # <r^2>
            expc_r_sq += np.vdot(r_n[n, :], r_n[n, :])  # <\vec{r}>^2

        spread = expc_rsq - expc_r_sq

        # if decomp:
        #     Omega_i = 0
        #     Omega_tilde = 0
        #     for k in k_idx_arr:
        #         for idx, b in enumerate(k_shell[0]):
        #             Omega_i += (1 / Nk) * w_b * n_states
        #             for n in range(n_states):
        #                 Omega_tilde += (
        #                     (1 / Nk) * w_b
        #                     * (-np.log(M[k][idx, n, n]).imag - np.vdot(b, r_n[n])) ** 2
        #                 )
        #                 for m in range(n_states):
        #                     Omega_i -= (1 / Nk) * w_b * abs(M[k][idx, m, n]) ** 2
        #                     if m != n:
        #                         Omega_tilde += (1 / Nk) * w_b * abs(M[k][idx, m, n]) ** 2

        #     return [spread, Omega_i, Omega_tilde], r_n, rsq_n

        # else:
        return spread, r_n, rsq_n
        
    return run()

In [72]:
spread_recip(lat_vecs, M)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function iadd>) found for signature:
 
 >>> iadd(readonly array(float64, 1d, C), array(float64, 1d, C))
 
There are 18 candidate implementations:
      - Of which 14 did not match due to:
      Overload of function 'iadd': File: <numerous>: Line N/A.
        With argument(s): '(readonly array(float64, 1d, C), array(float64, 1d, C))':
       No match.
      - Of which 2 did not match due to:
      Overload in function 'NumpyRulesInplaceArrayOperator.generic': File: numba/core/typing/npydecl.py: Line 243.
        With argument(s): '(readonly array(float64, 1d, C), array(float64, 1d, C))':
       Rejected as the implementation raised a specific error:
         AttributeError: 'NoneType' object has no attribute 'args'
  raised from /Users/treycole/miniforge3/envs/rutgers/lib/python3.10/site-packages/numba/core/typing/npydecl.py:254
      - Of which 2 did not match due to

In [51]:
@njit
def test():
    np.prod(np.array([1, 2, 3]))
    for i in range(10):
        x = np.log(np.random.rand(10,10))

In [52]:
test()